# 1. Data Loading and Cleaning

**Kristian Newell**

**Course: BrainStation Data Science**

**Next Notebook: 2. EDA**

The goal of my overall project is to be able to predict earthquake depth and magnitude through supervised learning. This will allow for pre-planning to lessen the economic burden and loss of life due to earthquakes each year.

The dataset used for this analysis is a public domain dataset created by the US Geological Survey and is available for download at [https://www.kaggle.com/usgs/earthquake-database]. This dataset is formatted as a CSV for easy access.

The data in this dataset is comprised of a record of the date, time, location, depth, magnitude, and source of every seismic event larger than a reported magnitude of 5.5 from 1965 to 2016. I opted not to web-scrape data to fill in the recent most 5 years as time did not permit 60 years of training data should be enough. 

Now that my dataset has been described and we know what features and observations we can expect, it is time to read in the CSV file to begin to look at the data.

The first step is going to be importing our standard tools for data loading, preprocessing, and cleaning.

In [1]:
# Loading in the usual toolkit required
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px

# Loading in datetime incase I want to manipulate the date and time features
import datetime

## Reading in the Data

I am now going to read in my data from the `csv` file using the pandas `read_csv` function and storing it as a DataFrame. After loading in the data I will be using `head()` method of DataFrames to view the first few lines of the DataFrame.

In [2]:
# Setting working directory
import os
os.chdir('C:/Users/Owner/Brainstation')

In [3]:
# Reading in my dataset
df=pd.read_csv('Capstone/earthquake_database.csv')

In [4]:
# Viewing the first few rows of data
df.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


## Data Cleaning and Processing

Now I am going to run an `info` method to inspect the columns of my new DataFrame.

In [5]:
# Checking the column names and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        23412 non-null  object 
 1   Time                        23412 non-null  object 
 2   Latitude                    23412 non-null  float64
 3   Longitude                   23412 non-null  float64
 4   Type                        23412 non-null  object 
 5   Depth                       23412 non-null  float64
 6   Depth Error                 4461 non-null   float64
 7   Depth Seismic Stations      7097 non-null   float64
 8   Magnitude                   23412 non-null  float64
 9   Magnitude Type              23409 non-null  object 
 10  Magnitude Error             327 non-null    float64
 11  Magnitude Seismic Stations  2564 non-null   float64
 12  Azimuthal Gap               7299 non-null   float64
 13  Horizontal Distance         160

Several of the columns are inconsequential to the model I would like to create, specifically: 

    *Depth Error 
    *Depth Seismic Stations 
    *Magnitude Type
    *Magnitude Error
    *Magnitude Seismic Stations
    *Azimuthal Gap
    *Horizontal Distance
    *Horizontal Error
    *Root Mean Square
    *ID
    *Source
    *Location Source
    *Magnitude Source
    *Status
As such I will be removing these columns from my working DataFrame.

I am choosing to remove these columns because they either contain a large amount of Nulls, like `Magnitude Error` or 

In [6]:
# Creating a new DataFrame without the unimportant rows
earthquake_df=df[['Date','Time','Type','Latitude','Longitude','Magnitude','Depth']]

I am inspecting the `Type` column of my DataFrame to ensure that all the rows of data I have included are for only naturally occurring seismic events.

In [7]:
# Checking the distribution of Type
earthquake_df['Type'].value_counts()

Earthquake           23232
Nuclear Explosion      175
Explosion                4
Rock Burst               1
Name: Type, dtype: int64

There are still some observances in the data that were caused by seismic events other than earthquakes, so I will be dropping these rows.

In [8]:
# Pulling out and dropping all rows that are not Earthquakes
earthquake_df.drop(earthquake_df.index.where(earthquake_df['Type']!='Earthquake').dropna(),inplace=True)

C:\Users\Owner\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
# Checking to see only earthquakes are left
earthquake_df.value_counts('Type')

Type
Earthquake    23232
dtype: int64

Now that I am sure that each observation in the data represents an earthquake, the `Type` column has become irrelevant, as such I will now drop it from the DataFrame.

In [10]:
# Dropping the type column as all rows are now type earthquake
earthquake_df.drop('Type',axis=1,inplace=True)

I now want to create a single datetime feature from the two features `Date` and `Time`.

In [11]:
# Commented out this cell because it throws errors

    # Converting date and time columns to a single datetime column
    # pd.to_datetime(earthquake_df.Date + ' '+ earthquake_df.Time)

# ParserError: Unknown string format: 1975-02-23T02:58:41.000Z 1975-02-23T02:58:41.000Z

There was an issue adding the two features together. The error indicates that the problematic row includes a "T" somewhere. Instead of combining the two columns and casting type to `datetime` in one line, I will need to do these steps piecewise to fix the issue.

In [12]:
# Combining the columns without changing type
earthquake_df['DateAndTime'] = earthquake_df['Date'].str.cat(earthquake_df['Time'],sep=" ")

<ipython-input-12-a8ec06989b16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earthquake_df['DateAndTime'] = earthquake_df['Date'].str.cat(earthquake_df['Time'],sep=" ")


In [13]:
# Checking to see which rows were throwing errors because they include a T
earthquake_df[earthquake_df['DateAndTime'].str.contains('T')]

,Date,Time,Latitude,Longitude,Magnitude,Depth,DateAndTime
3378,1975-02-23T02:58:41.000Z,1975-02-23T02:58:41.000Z,8.017,124.075,5.6,623.0,1975-02-23T02:58:41.000Z 1975-02-23T02:58:41.000Z
7512,1985-04-28T02:53:41.530Z,1985-04-28T02:53:41.530Z,-32.998,-71.766,5.6,33.0,1985-04-28T02:53:41.530Z 1985-04-28T02:53:41.530Z
20650,2011-03-13T02:23:34.520Z,2011-03-13T02:23:34.520Z,36.344,142.344,5.8,10.1,2011-03-13T02:23:34.520Z 2011-03-13T02:23:34.520Z


I have filtered out the rows that had a "T" present in `Date` or `Time` and it appears to be only 3 rows out of 23,232. Because these three error rows represent only 0.01% of the data, I am confident that the removal of these rows will not affect the outcome of the model. As such I will be dropping them so that I can convert the `DateAndTime` colum to data type datetime.

In [14]:
# Dropping these rows
earthquake_df.drop([3378,7512,20650], axis=0, inplace= True)

C:\Users\Owner\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
# Converting DateAndTime
earthquake_df['DateAndTime']=pd.to_datetime(earthquake_df['DateAndTime'])

<ipython-input-15-c36f2402b31a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earthquake_df['DateAndTime']=pd.to_datetime(earthquake_df['DateAndTime'])


In [16]:
# Checking to see that the column data type has been changed
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23229 entries, 0 to 23411
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         23229 non-null  object        
 1   Time         23229 non-null  object        
 2   Latitude     23229 non-null  float64       
 3   Longitude    23229 non-null  float64       
 4   Magnitude    23229 non-null  float64       
 5   Depth        23229 non-null  float64       
 6   DateAndTime  23229 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 1.4+ MB


In [17]:
# Dropping the original time and date
earthquake_df.drop(['Date','Time'],axis=1,inplace=True)

The `Date` and `Time` columns have been combined into a single column `DateAndTime` of type datetime.

The data has been successfully loaded, preprocessed and cleaned; it is now ready for EDA. As such, I am going to export my cleaned DataFrame so that I may use it in my next notebook **2. EDA (Exploratory Data Analysis**. In this next workbook I will be visualizing my data as well as determining if there are any intra-feature relationships.

In [18]:
# Exporting file to CSV now that it has been cleaned
earthquake_df.to_csv('C:/Users/Owner/Brainstation/Capstone/cleaned_df.csv',index=False)